# Using LLM to Analyze Sense of Place and Make Storymaps

### Author: Bo Zhao
### Date: May 10, 2025

This Colab notebook demonstrates how to use a large language model (LLM), specifically Gemini Flash 2.0 or Gemini, to perform spatial narrative extraction and sense of place analysis from a chapter of a geographic text. Focused on Seattle’s Capitol Hill, the workflow includes:

- Extracting landmark data from a PDF using LLM prompts

- Structuring that data into a CSV with geolocation and narrative attributes

- Converting the CSV into a GeoJSON file

- Visualizing locations on an interactive Folium map

- Generating a scrollable, HTML-based story map powered by MapLibre GL JS

Each location is enriched with semantic information (e.g., place identity, memory, rhetorical framing) and rendered with numbered markers and images. The entire process is automated and modular, designed for rapid experimentation with LLM-powered geo-narrative workflows.

In [108]:
# --- 1. Install dependencies ---
!pip install google-generativeai

In [109]:
# --- 2. Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
# --- 3. Set Gemini API Key ---
import os
import google.generativeai as genai
import json
import pandas as pd

In [113]:
genai.configure(api_key='')  # please input your API Key. You can apply for a Gemini Key from Google.

In [115]:
# --- 4. Upload PDF binary content ---
pdf_path = "/content/drive/MyDrive/projects/GEOG404/Seattle_Walks_Ch13.pdf"
with open(pdf_path, "rb") as f:
    pdf_content = f.read()

In [116]:
# --- 5. Prompt for extracting structured spatial data ---
prompt_text = '''
Read this PDF file and extract all major landmarks described by the author along the Capitol Hill route.
For each landmark, return the result as a comma-separated table (CSV format) with the following columns:

place_id,name,location_description,latitude,longitude,relative_distance,sense_of_place_type,sense_explanation,rhetorical_usage

Example:
place_id,name,location_description,latitude,longitude,relative_distance,sense_of_place_type,sense_explanation,rhetorical_usage
1,Volunteer Park Conservatory,"Located inside Volunteer Park near E Prospect St",47.6303,-122.3145,start,belonging,"Described as a community haven surrounded by nature","The greenhouse 'greets you like an old friend'"
2,Asian Art Museum,"Adjacent to the conservatory, facing west",47.6309,-122.3148,short walk,memory,"Connects to shared cultural memory of Asian heritage","Buildings are 'silent narrators of migration'"

Do not include any notes or explanation. Make sure it has lat long, even it was estimated. Output only the CSV content, starting with the header line.
'''

In [117]:
# model = genai.GenerativeModel("gemini-2.0-flash") #Gemini Flash
model = genai.GenerativeModel("models/gemini-2.5-pro-exp-03-25") #Gemini Pro
response = model.generate_content(
    [
        {"text": prompt_text},
        {"mime_type": "application/pdf", "data": pdf_content},
    ]
)

In [118]:
# --- 6. Parse CSV and save as file ---
csv_text = response.text.strip()

# Save CSV to file
csv_path = "capitol_hill.csv"
with open(csv_path, "w", encoding="utf-8") as f:
    f.write(csv_text)

print("\n✅ CSV file saved as 'capitol_hill.csv'\nDownload it below:")
from google.colab import files
files.download(csv_path)


✅ CSV file saved as 'capitol_hill.csv'
Download it below:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [119]:
# --- 7. Convert CSV to GeoJSON ---
df = pd.read_csv(csv_path, quotechar='"', skip_blank_lines=True, on_bad_lines='skip')

features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["longitude"], row["latitude"]]
        },
        "properties": row.drop(["latitude", "longitude"]).to_dict()
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

geojson_path = "capitol_hill.geojson"
with open(geojson_path, "w", encoding="utf-8") as f:
    import json
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("\n✅ GeoJSON file saved as 'capitol_hill.geojson'\nDownload it below:")
files.download(geojson_path)



✅ GeoJSON file saved as 'capitol_hill.geojson'
Download it below:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [120]:
df = pd.read_csv(csv_path, quotechar='"', skip_blank_lines=True, on_bad_lines='skip')
# --- Optional: Display interactive map in Colab ---
import folium

# Center the map on the average location
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

for _, row in df.iterrows():
    popup_text = f"""
    <b>{row['place_id']}. {row['name']}</b><br>
    <i>{row['location_description']}</i><br>
    <b>Sense:</b> {row['sense_of_place_type']}<br>
    <b>Why:</b> {row['sense_explanation']}<br>
    <b>Style:</b> {row['rhetorical_usage']}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['name'],
        icon=folium.DivIcon(html=f"""
          <div style="
              background-color: #2c3e50;
              color: white;
              border-radius: 50%;
              width: 28px;
              height: 28px;
              text-align: center;
              line-height: 28px;
              font-size: 12px;
              font-weight: bold;
              box-shadow: 0 0 3px rgba(0,0,0,0.5);">
              {int(row['place_id'])}
          </div>
      """)

    ).add_to(m)

# Display map in output
m

In [101]:
# model = genai.GenerativeModel("gemini-2.0-flash") #Gemini Flash
model = genai.GenerativeModel("models/gemini-2.5-pro-exp-03-25") #Gemini Pro

# --- 8. Let Gemini generate MapLibre StoryMap HTML ---
prompt_html = '''
Create an HTML file called index.html that uses the latest version of MapLibre GL JS (https://maplibre.org/maplibre-gl-js/docs/) to display a story map of a walk through Seattle's Capitol Hill.

Use the GeoJSON file 'capitol_hill.geojson' (in the same folder) as the data source. Each feature in the file should be shown as a numbered marker (using place_id) on the map.

Use the following example GeoJSON feature to guide the structure:
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [-122.3121, 47.6324]
  },
  "properties": {
    "place_id": 1,
    "name": "Louisa Boren Park",
    "location_description": "Start of the walk, located at 15th Avenue E and E Galer St/E Garfield St",
    "relative_distance": "start",
    "sense_of_place_type": "Memory / Commemoration",
    "sense_explanation": "Honors Louisa Boren, one of Seattle's earliest citizens and founders; contains historic trees moved from Denny Park.",
    "rhetorical_usage": "'hidden little treasure on Capitol Hill.'"
  }
}

The layout should include:
- A clean, full-screen layout with a fixed MapLibre map on the right side,
- A scrollable sidebar on the left with one section per landmark.

Each section should show:
- the place name,
- location_description,
- sense_of_place_type and explanation,
- rhetorical_usage,
- and an image pulled from Unsplash (or a placeholder using the place name).

Use a commonly available open street map-style basemap that does not require an API key, but do not hardcode a specific one—leave flexibility for the user to configure it.

As users click or scroll manually, the map should fly to the corresponding feature.

Use best practices for performance, accessibility, and modern HTML/CSS. Return only the complete HTML code.
'''

response_html = model.generate_content(prompt_html)

# Save generated HTML to file
html_path = "index.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write(response_html.text)

print("\n✅ index.html generated by Gemini and saved. \nDownload it below:")
files.download(html_path)



✅ index.html generated by Gemini and saved. 
Download it below:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>